# 4. Import data from MS Excel

Brightway2.5 has a series of options for data import and export that you are invited to read about and try, they are on the official website and notebook. 

However, you can also developed your own importer, that fits with your workflow. For example, the file `lci_to_bw2.py` includes a code to convert a properly formatted csv file into a Brightway2.5 database dict. You need to install the Python Data Analysis Library [pandas](https://pandas.pydata.org/) to make it work (within your virtual environment, run `conda install pandas` or `pip install pandas` if you are not using conda). 

How does this importer work? 

1. Prepare your inventory in MS Excel using the template. See the example file _test\_db\_excel\_w\_ecoinvent.xlsx_
2. Save the relevant MS Excel sheet as .csv file, see the example file _test\_db\_excel\_w\_ecoinvent.csv_
3. Import the module in your script with the command `from lci_to_bw2 import *` 
4. Import the .csv file as a dataframe with the pandas function `.read_csv()`. Clean it up for unnecessary columns.
5. Convert the dataframe into a dict using the function `lci_to_bw2()`
6. Save the dict as a Brightway2.5 database in the usual way i.e. using Brightway's `Database()`and `.write()` functions.

**NOTE:** this importer contains no automated tests so you need to make sure manually that the excel and csv files are in good order.

See an example below.

In [1]:
# Import brightway2.5 packages
import bw2calc as bc
import bw2data as bd
import pandas as pd
import numpy as np
from lci_to_bw2 import * # import all the functions of this module

In [2]:
mydb = pd.read_csv('test_db_excel_w_ecoinvent.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydb.head()

,Activity database,Activity code,Activity name,Activity unit,Activity type,Exchange database,Exchange input,Exchange amount,Exchange unit,Exchange type,Exchange uncertainty type,Exchange loc,Exchange scale,Exchange negative,Notes
0,exldb,Electricity production,Electricity production,﻿kilowatt hour,process,exldb,Electricity production,10.0,kilowatt hour,production,NaN,NaN,NaN,NaN,NaN
1,exldb,Electricity production,Electricity production,﻿kilowatt hour,process,exldb,Fuel production,2.0,kilogram,technosphere,2.0,1.0,1.01,False,NaN
2,exldb,Electricity production,Electricity production,﻿kilowatt hour,process,biosphere3,349b29d1-3e58-4c66-98b9-9d1a076efd2e,1.0,kilogram,biosphere,2.0,1.0,1.01,False,Carbon dioxide
3,exldb,Electricity production,Electricity production,﻿kilowatt hour,process,exldb,Sulphur dioxide,0.1,kilogram,biosphere,0.0,2.0,2.00,True,NaN
4,exldb,Fuel production,Fuel production,kilogram,process,exldb,Fuel production,100.0,kilogram,production,NaN,NaN,NaN,NaN,NaN


In [3]:
# clean up a bit
mydb = mydb.drop('Notes', axis=1)  # remove the columns not needed
mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default
mydb.head()

,Activity database,Activity code,Activity name,Activity unit,Activity type,Exchange database,Exchange input,Exchange amount,Exchange unit,Exchange type,Exchange uncertainty type,Exchange loc,Exchange scale,Exchange negative
0,exldb,Electricity production,Electricity production,﻿kilowatt hour,process,exldb,Electricity production,10.0,kilowatt hour,production,0,NaN,NaN,NaN
1,exldb,Electricity production,Electricity production,﻿kilowatt hour,process,exldb,Fuel production,2.0,kilogram,technosphere,2,1.0,1.01,False
2,exldb,Electricity production,Electricity production,﻿kilowatt hour,process,biosphere3,349b29d1-3e58-4c66-98b9-9d1a076efd2e,1.0,kilogram,biosphere,2,1.0,1.01,False
3,exldb,Electricity production,Electricity production,﻿kilowatt hour,process,exldb,Sulphur dioxide,0.1,kilogram,biosphere,0,2.0,2.00,True
4,exldb,Fuel production,Fuel production,kilogram,process,exldb,Fuel production,100.0,kilogram,production,0,NaN,NaN,NaN


In [4]:
# Create a dict that can be written as database
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module
bw2_db

{('exldb', 'Electricity production'): {'name': 'Electricity production',
  'unit': '\ufeffkilowatt hour',
  'type': 'process',
  'exchanges': [{'input': ('exldb', 'Electricity production'),
    'amount': 10.0,
    'unit': 'kilowatt hour',
    'type': 'production',
    'uncertainty type': 0},
   {'input': ('exldb', 'Fuel production'),
    'amount': 2.0,
    'unit': 'kilogram',
    'type': 'technosphere',
    'uncertainty type': 2,
    'loc': 1.0,
    'scale': 1.01,
    'negative': False},
   {'input': ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'),
    'amount': 1.0,
    'unit': 'kilogram',
    'type': 'biosphere',
    'uncertainty type': 2,
    'loc': 1.0,
    'scale': 1.01,
    'negative': False},
   {'input': ('exldb', 'Sulphur dioxide'),
    'amount': 0.1,
    'unit': 'kilogram',
    'type': 'biosphere',
    'uncertainty type': 0,
    'loc': 2.0,
    'scale': 2.0,
    'negative': True}]},
 ('exldb', 'Fuel production'): {'name': 'Fuel production',
  'unit': 'kilogram',
  'typ

In [5]:
bd.projects.set_current('advlca25') # Find a project where there is ecoinvent 3.6 conseq
bd.databases

Databases dictionary with 4 object(s):
	biosphere3
	ecoinvent 3.9.1 conseq
	testbiosphere
	testdb

Time to write the data on a database. 

Important: 

- The database **name should be the same** as in the excel file...

- make sure you **shut down** all other notebooks using **the same bw project** before you run this. Only one user at the time can write on a database. Otherwise you'll get a "Database locked" error.

In [6]:
t_db = bd.Database('exldb') # it works because the database name in the excel file is the same
# shut down all other notebooks using the same project
t_db.write(bw2_db)
# Error: "Not able to determine geocollections for all datasets. This database is not ready for regionalization." 
# This is because the "activity" data don't have 'location' key-value pair, but it is fine, you can add it or not.

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 2514.19it/s]

Vacuuming database 


In [7]:
bd.databases # It worked

Databases dictionary with 5 object(s):
	biosphere3
	ecoinvent 3.9.1 conseq
	exldb
	testbiosphere
	testdb

Give a look at your imported database

In [8]:
[print(act) for act in t_db]  # check more stuff 
print('---------')
[[print(act, exc) for exc in list(act.exchanges())]for act in t_db]  # check more stuff 
print('---------')
[[print(exc.uncertainty) for exc in list(act.exchanges())]for act in t_db]  # check more stuff


'Electricity production' (﻿kilowatt hour, None, None)
'Crude oil' (kilogram, None, None)
'Sulphur dioxide' (kilogram, None, None)
'Fuel production' (kilogram, None, None)
---------
'Electricity production' (﻿kilowatt hour, None, None) Exchange: 10.0 ﻿kilowatt hour 'Electricity production' (﻿kilowatt hour, None, None) to 'Electricity production' (﻿kilowatt hour, None, None)>
'Electricity production' (﻿kilowatt hour, None, None) Exchange: 2.0 kilogram 'Fuel production' (kilogram, None, None) to 'Electricity production' (﻿kilowatt hour, None, None)>
'Electricity production' (﻿kilowatt hour, None, None) Exchange: 1.0 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'Electricity production' (﻿kilowatt hour, None, None)>
'Electricity production' (﻿kilowatt hour, None, None) Exchange: 0.1 kilogram 'Sulphur dioxide' (kilogram, None, None) to 'Electricity production' (﻿kilowatt hour, None, None)>
'Fuel production' (kilogram, None, None) Exchange: 100.0 kilogram 'Fuel production' 

[[None, None, None, None, None], [], [], [None, None, None, None]]

In [9]:
myact = bd.Database("exldb").get('Fuel production')
exchanges = list(myact.exchanges())

Let's check if calculations work

In [10]:
mymethod = ('IPCC 2013', 'climate change', 'global warming potential (GWP100)')
el = t_db.get("Electricity production")
functional_unit = {el: 1000}
lca = bc.LCA(functional_unit, mymethod)
lca.lci()
lca.lcia()
print(lca.score)

324.1823668256106


# Question
Do you think this results includes all emissions you have imported from the excel file?

# Group exercise 

### (This is for the portfolio you don't need to do it now, we'll start it during the online lecture) 
Prepare your own product system in excel, linked to biosphere3 and ecoinvent, and import it. Run calculations to see if it works as expected. Send all the code and data to another group and see if they can reproduce your results, in that case, the exercise will be a success. Get feedback from other group on your code and what difficulties they had in reading and running it.